In [1]:
import polars as pl
import altair as alt
from datetime import datetime

In [2]:
gdp = "../data/gdp.csv"
credit_g = "../data/credit_g.csv"
ir = "../data/ir.csv"
m2 = "../data/m2.csv"
real_est_p = "../data/real_est_p.csv"
unemp_r = "../data/unemp_r.csv"
monetary_base = "../data/monetary_base.csv"

In [3]:
gdp_df = pl.read_csv(gdp, try_parse_dates=True)
credit_g_df = pl.read_csv(credit_g, try_parse_dates=True)
ir_df = pl.read_csv(ir, try_parse_dates=True)
m2_df = pl.read_csv(m2, try_parse_dates=True)
real_est_p_df = pl.read_csv(real_est_p, try_parse_dates=True)
unemp_r_df = pl.read_csv(unemp_r, try_parse_dates=True)
monetary_b_df = pl.read_csv(monetary_base, try_parse_dates=True)

In [16]:
gdp_out = gdp_df.select((pl.col("GDP (current US$)") / 1000000000).cast(pl.Int64).alias("gdp_current_dol_in_bln"),
                pl.col("Year").alias("year")).filter(pl.col("year") >= 2019)

credit_g_out = credit_g_df.select(pl.col("Year").alias("year"),
                                 pl.col("Domestic credit to private sector (% of GDP)").alias("debt_to_gdp")
                                 ).filter(pl.col("year") >= 2019)

ir_out = ir_df.select(pl.col("Period").str.to_date("%YM%m").alias("year"),
                     pl.col("Interest rate. %").alias("interest_rate_percent")
                     ).filter(pl.col("year") >= datetime(2019, 1, 1))

m2_out = m2_df.select(pl.col("Date").str.to_date("%Y/%m").alias("year"),
                     (pl.col("M2/Average Amounts Outstanding/Money Stock") / 1000000).alias("m2_money_supply")
                     )

real_est_p_out = real_est_p_df.select(pl.col("Date").alias("year"),
                                     pl.col("Value").alias("value")
                                     ).filter(pl.col("year") >= datetime(2019, 1, 1))

unemp_r_out = unemp_r_df.select(pl.col("Year").alias("year"),
                                pl.col("Unemployment, total (% of total labor force) (national estimate)").alias("unemployment_rate")
                               ).filter(pl.col("year") >= 2019)

monetary_b_out = monetary_b_df.select(pl.col("period").str.to_date("%Y/%m").alias("date"),
                                     (pl.col("value_in_millions") / 1000000)
                                     )

In [5]:
gdp_chart = (
    (gdp_out.plot.line(
        x=alt.X("year", axis = alt.Axis(format = "d")), 
        y=alt.Y("gdp_current_dol_in_bln", scale=alt.Scale(zero=True))
                      )
    )
        .properties(width=400)
        .configure_scale(zero=False)
)

gdp_chart

alt.Chart(...)

In [6]:
credit_g_chart = (
    (credit_g_out.plot.line(
        x=alt.X("year", axis = alt.Axis(format = "d")), 
        y=alt.Y("debt_to_gdp", scale=alt.Scale(zero=True))
                      )
    )
        .properties(width=400)
        .configure_scale(zero=False)
)

credit_g_chart

alt.Chart(...)

In [7]:
base = (
    ir_out.plot.line(
        x="year", 
        y=alt.Y("interest_rate_percent", scale=alt.Scale(zero=True, padding=1))
                      )
)

zero_line = alt.Chart().mark_rule(
    color='gray',
    strokeDash=[8, 8],
    opacity=0.8
).encode(y=alt.datum(0))

ir_chart = (
    (base + zero_line)
    .properties(width=400)
    .configure_scale(zero=False)
)

ir_chart

alt.LayerChart(...)

In [8]:
m2_chart = (
    (m2_out.plot.line(
        x="year", 
        y=alt.Y("m2_money_supply", scale=alt.Scale(zero=True))
                      )
    )
        .properties(width=400)
        .configure_scale(zero=False)
)

m2_chart

alt.Chart(...)

In [9]:
base = (
    (real_est_p_out.plot.line(
        x="year", 
        y=alt.Y("value", scale = alt.Scale(zero = True))
                      )
    )
)

zero_line = alt.Chart().mark_rule(
    color='gray',
    strokeDash=[8, 8],
    opacity=0.8
).encode(y=alt.datum(100))

real_est_p_chart = (
    (base + zero_line)
    .properties(width=400)
    .configure_scale(zero=False)
)

real_est_p_chart

alt.LayerChart(...)

In [10]:
unemp_r_chart = (
    (unemp_r_out.plot.line(
        x=alt.X("year", axis=alt.Axis(format="d")), 
        y=alt.Y("unemployment_rate", scale=alt.Scale(zero=True))
                      )
    )
        .properties(width=400)
        .configure_scale(zero=False)
)

unemp_r_chart

alt.Chart(...)

In [17]:
monetary_b_chart = (
    (monetary_b_out.plot.line(
        x=alt.X("date:T", axis=alt.Axis(format="%Y")), 
        y=alt.Y("value_in_millions", scale=alt.Scale(zero=True))
                      )
    )
        .properties(width=400)
        .configure_scale(zero=False)
)

monetary_b_chart

alt.Chart(...)

In [18]:
gdp_chart.save('../charts/gdp_chart_5y.png')
credit_g_chart.save('../charts/credit_g_chart_5y.png')
ir_chart.save('../charts/ir_chart_5y.png')
m2_chart.save('../charts/m2_chart.png')
real_est_p_chart.save('../charts/real_est_p_chart_5y.png')
unemp_r_chart.save('../charts/unemp_r_chart_5y.png')
monetary_b_chart.save('../charts/monetary_b_chart.png')